### Tesseract本体とPythonラッパーをインストールします。

In [ ]:
!sudo apt-get purge tesseract-ocr libtesseract-dev tesseract-ocr-jpn
!sudo apt-get install tesseract-ocr libtesseract-dev tesseract-ocr-jpn
!pip install pytesseract

### Githubより、必要なものを一式をダウンロードします。

In [ ]:
%cd /content/
!git clone https://github.com/MamiTakata/aiocr-test.git
%cd /content/aiocr-test

### 以下プログラムを実行します。

In [ ]:
# パッケージをインポート
import numpy as np
import cv2
import  imutils
import sys
import pytesseract
from pytesseract import Output
import time
import matplotlib.pyplot as plt

In [ ]:
# 定数を定義
IMAGE_PATH = "images/car.jpg"

In [ ]:
# 画像表示用
def plt_imshow(title, image):
	image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
	plt.imshow(image)
	plt.title(title)
	plt.grid(False)
	plt.show()

In [ ]:
# 画像読み込みして表示
image = cv2.imread(IMAGE_PATH)
plt_imshow("Original Image", image)

In [ ]:
# Cannyエッジ検出
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5,), 0)
edged = cv2.Canny(blurred, 75, 200)
plt_imshow("Canny Edges Image", edged)

# 輪郭検出
contours = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
contours = imutils.grab_contours(contours)
contours = sorted(contours, key=cv2.contourArea, reverse=True)[:30]

In [ ]:
# 取得した輪郭から形状の近似を行い長方形を探す（無理やりだがそれをナンバープレートとする）
number_contours = None
for c in contours:
  perimeter = cv2.arcLength(c, True)
  approx = cv2.approxPolyDP(c, 0.02 * perimeter, True)
  if len(approx) == 4:
      number_contours = approx
      break

print(number_contours)

In [ ]:
# ナンバープレート以外はマスクする
if number_contours is not None:
  mask = np.zeros(gray.shape,dtype="uint8")
  image_plate = cv2.drawContours(mask,[number_contours],0,255,-1)
  image_plate = cv2.bitwise_and(gray,gray,mask=mask)
  plt_imshow("License plate Image",image_plate)

In [ ]:


# ナンバープレート画像からテキスト検出
# config = ('-l jpn --oem 3 --psm 6 -c tessedit_char_whitelist="0123456789-あいうえおかきくけこさしすせそたちつてとなにぬねのはひふへほまみむめもやゆよらりるれろわをん" ')
config = ('-l jpn --oem 3 --psm 6  ')
text = pytesseract.image_to_string(image_plate, config=config)

if text:
  print(text)
else:
  print("Could not read license plate.")